## Homework 4 ##

### Question 2: Min-Cut problem formulation as an LP problem

 The minimum cut problem requires identifying the minimal set of edges that, when removed, completely separates the source and sink vertices, creating two non-overlapping partitions in the graph. In other words, given
any subset S of nodes with source node $s \in S$, let T be the set of the remaining nodes. The cut (S, T ) is the set of edges $ e = (u, v)$ with $u \in S$ and $v \in  T$. It is so-called because removing all those edges in the cut would cut the flow from s to t. Let c(e), $e \in E$ be the capacity of the edge, i.e., the maximum amount of commodity that one can push through the edge. Let $x_k$, $\forall$ $k \in V$ be a variable corresponding to the vertices and let $y_e$,  $\forall$ $e =(u,v) \in E$ be a variable corresponding to the edges. The min-cut problem can be formulated as the following:

\begin{align*}
\text{minimize} \quad & \sum_{e \in E} c(e) y_e \\
\text{subject to} \quad & x_u - x_v + y_e \geq 0 \quad \forall e=(u, v) \in E \\
& -x_s + x_t \geq 1 \\
& x_v \in \mathbb{R} \quad \forall v \in V \\
& y_e \geq 0 \quad \forall e \in E
\end{align*}

For the directed graph given below, solve the min-cut problem using Pyomo

In [ ]:
# import image module
from IPython.display import Image

# get the image
Image(url="fig1.png", width=500, height=300)

In [1]:
# Including Libraries

from pyomo.environ import *
import numpy as np
import pprint

In [2]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 124947 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

In [3]:
# Create a Pyomo Model
model = ConcreteModel()

# Define sets

model.v = RangeSet(0,5)

edges = [(0,1), (0,2), (1,2), (1,3), (2,1), (2,4), (3,2), (3,5), (4,3), (4,5)] # Edge tuples
model.edges = Set(initialize = edges)

# Define parameters

cap_list = [16, 13, 10, 12, 4, 14, 9, 20, 7, 4]
capacities = {(i, j): cap_list[idx] for idx, (i, j) in enumerate(model.edges)}

model.cap = Param(model.edges, initialize = capacities)

# Define variables

model.x = Var(model.v)

model.y = Var(model.edges, domain = NonNegativeReals)

In [4]:
# Define objective

model.obj = Objective(expr = sum(model.cap[(i,j)]*model.y[(i,j)] for i,j in model.edges), sense = minimize)

# Define constraints

def const_rule1(model,i,j):
    return(model.x[i] - model.x[j] + model.y[(i,j)] >= 0)

model.const1 = Constraint(model.edges, rule = const_rule1)

model.const2 = Constraint(expr = -model.x[0] + model.x[5] >=1)

In [7]:
# Solve the problem

solver = SolverFactory('glpk')
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 23.0, 'Upper bound': 23.0, 'Number of objectives': 1, 'Number of constraints': 11, 'Number of variables': 16, 'Number of nonzeros': 32, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.007284879684448242}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [8]:
# printing the result

print("The optimal objective value is:", model.obj())

print("\nPrinting x variable values:")
for i in model.v:
    print("x",[i], ":", model.x[i].value)

print("\nPrinting y variable values:")
for i,j in model.edges:
    print("y",[i,j], ":", model.y[i,j].value)


The optimal objective value is: 23.0

Printing x variable values:
x [0] : 0.0
x [1] : 0.0
x [2] : 0.0
x [3] : 1.0
x [4] : 0.0
x [5] : 1.0

Printing y variable values:
y [0, 1] : 0.0
y [0, 2] : 0.0
y [1, 2] : 0.0
y [1, 3] : 1.0
y [2, 1] : 0.0
y [2, 4] : 0.0
y [3, 2] : 0.0
y [3, 5] : 0.0
y [4, 3] : 1.0
y [4, 5] : 1.0
